In [1]:
import msoffcrypto as mso

LOCAL_PATH = r"C:\\Users\\zls\\Desktop\\{}"
file_name = "2021.docx"
doc_path = LOCAL_PATH.format(str(file_name))

encrypted = open(doc_path, "rb")
file = mso.OfficeFile(encrypted)
file.load_key(password="13148422")

with open(LOCAL_PATH.format(str("decrypted.docx")), "wb") as f:
    file.decrypt(f)


encrypted.close()

In [4]:
from docx import Document
document = Document(LOCAL_PATH.format(str("decrypted.docx")))

for p in document.paragraphs:
    print(p.text)
    print(type(p.text))

2021
<class 'str'>
Quite a surprise to find him still a postdoc.
<class 'str'>


In [5]:
# 不传参的函数
def logger(func):
    def wrapper(*args, **kwargs):
        print('Prepare to execute: {} function.'.format(func.__name__))

        func(*args, **kwargs)

        print('Executed, now closing.')
    return wrapper

@logger
def add(x, y):
    print("{} + {} = {}".format(x, y, x+y))

add(200, 50)

Prepare to execute: add function.
200 + 50 = 250
Executed, now closing.


In [7]:
# 带传参的装饰器
def say_hello(country):
    def wrapper(func):
        def deco(*args, **kwargs):
            if country == 'china':
                print('您好！')
            elif country == 'america':
                print('Hello！')
            else:
                return
            
            func(*args, **kwargs)
        return deco
    return wrapper

@say_hello("china")
def xiaoming():
    pass

@say_hello("america")
def jack():
    pass

xiaoming()
print('=='*30)
jack()

您好！
Hello！


In [9]:
# 不带参数的类装饰器

class logger(object):
    def __init__(self, func): # init传入函数
        self.func = func

    def __call__(self, *args, **kwargs): # 实现装饰逻辑
        print("[INFO]: the function {func}() is running...".format(func=self.func.__name__))
        return self.func(*args, **kwargs)

@logger
def say(something):
    print("Say {}".format(something))

say("WHAT!!")

[INFO]: the function say() is running...
Say WHAT!!


In [12]:
# 带传参的类装饰器有所不同

class logger(object):
    def __init__(self, level = 'INFO'): # init不再传函数，改为传参
        self.level = level
    
    def __call__(self, func): # 传函数
        def wrapper(*args, **kwargs):
            print("[{level}]: the function {func}() is running...".format(level=self.level, func=func.__name__))
            func(*args, **kwargs)
        return wrapper # 返回函数

@logger(level='WARNING')
def say(something):
    print("Say {}".format(something))

say("Something Wrong.")

[WARNING]: the function say() is running...
Say Something Wrong.


In [17]:
# 使用functools里的偏函数

import time
import functools

class DelayFunc:
    def __init__(self, duration, func): # init同时传参和函数
        self.duration = duration
        self.func = func

    def __call__(self, *args, **kwargs):
        print(f'Wait for {self.duration} seconds...')
        time.sleep(self.duration)
        return self.func(*args, **kwargs)

    def emerg_call(self, *args, **kwargs):
        print('Call Immediately!')
        return self.func(*args, **kwargs)

def delay(duration):
    """
    装饰器：推迟某个函数的执行
    同时也有emerg_call立刻执行的方法
    """
    # 此处为了避免定义额外函数，直接使用functools.partial帮助构造DelayFunc实例
    return functools.partial(DelayFunc, duration)

@delay(duration=2)
def add(a, b):
    return a+b

add(3,5)
add.func

Wait for 2 seconds...


<function __main__.add(a, b)>

In [20]:
# 装饰类的装饰器

instances = {}
def singleton(cls):
    def get_instance(*args, **kwargs):
        cls_name = cls.__name__
        print('Name Acquired.')
        if not cls.__name__ in instances:
            print('Not retrieved')
            instance = cls(*args, **kwargs)
            instances[cls_name] = instance
        return instances[cls_name]
    return get_instance

@singleton
class User:
    _instance = None

    def __init__(self, name):
        print('Initializing User')
        self.name = name

u1 = User('A')

Name Acquired.
Not retrieved
Initializing User
